In [14]:
from tensorflow.keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('weights.best.h5')

In [15]:
import os 

original_dataset_dir = './dataset/dogs_and_cats'
test_dir = os.path.join(original_dataset_dir, "test")
filenames = [filename for filename in os.listdir(test_dir)]

filenames[:5]

['4224.jpg', '9426.jpg', '4955.jpg', '9847.jpg', '9993.jpg']

In [16]:
len(filenames)

12500

In [17]:
import pandas as pd

data = {"filename": filenames, "label": [0]*len(filenames)}

test_df = pd.DataFrame(data)

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_generator = ImageDataGenerator(
    rescale=1./255)


test_datagenerator = image_generator.flow_from_dataframe(
  test_df, test_dir, y_col=None, target_size=(150,150),batch_size=20, class_mode=None)

Found 12500 images.


In [19]:
y_probabilities = model.predict_generator(test_datagenerator)

In [20]:
y_probabilities

array([[0.        ],
       [0.9999999 ],
       [1.        ],
       ...,
       [1.        ],
       [1.        ],
       [0.25391638]], dtype=float32)

In [21]:
pred = (y_probabilities > 0.5).flatten() + 0

In [22]:
data = {"id": [filename.split(".")[0] for filename in filenames], "label": y_probabilities.flatten()}

result_df = pd.DataFrame(data)

In [23]:
result_df.set_index("id").to_csv("submission.csv")

In [24]:
!kaggle competitions submit -f submission.csv -m "VGG with Adam" dogs-vs-cats-redux-kernels-edition

100%|████████████████████████████████████████| 128k/128k [00:03<00:00, 35.8kB/s]
Successfully submitted to Dogs vs. Cats Redux: Kernels Edition

In [13]:
pred

array([0, 1, 0, ..., 1, 0, 1])